# Shapes in CNNS

## Shapes in the Feature Extractors

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten


N, n_H, n_W, n_C = 32, 28, 28, 3
n_conv_filter = 5
batch_size=32
k_size=3 # f (kernel_size)
pool_size, pool_strides = 2,2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

conv1 = Conv2D(filters=n_conv_filter, kernel_size=k_size,
              padding='same', activation='relu')
conv1_pool = MaxPooling2D(pool_size=pool_size, 
                          strides=pool_strides
                          )

conv2 = Conv2D(filters=n_conv_filter, 
               kernel_size=k_size,
               padding='same', 
               activation='relu')
conv2_pool = MaxPooling2D(pool_size=pool_size, 
                          strides=pool_strides
                          )

flatten = Flatten()

print('first input\n', x.shape)

x = conv1(x)
W,B = conv1.get_weights()
print('after conv1\n', x.shape)
print('W,B : {}/{}'.format(W.shape, B.shape))
x = conv1_pool(x)
print('after conv1 pool\n', x.shape) # 2x2 pooling을 하여 shape가 반으로 줄어듬. 

x = conv2(x)
W,B = conv2.get_weights()
print('after conv2\n', x.shape)
print('W,B : {}/{}'.format(W.shape, B.shape))

x = conv2_pool(x)
print('after conv2 pool\n', x.shape)

x = flatten(x)
print('after flatten\n', x.shape) 

first input
 (32, 28, 28, 3)
after conv1
 (32, 28, 28, 5)
W,B : (3, 3, 3, 5)/(5,)
after conv1 pool
 (32, 14, 14, 5)
after conv2
 (32, 14, 14, 5)
W,B : (3, 3, 5, 5)/(5,)
after conv2 pool
 (32, 7, 7, 5)
after flatten
 (32, 245)


245

## Shapes in the Classifier

In [23]:
from tensorflow.keras.layers import Dense

n_neurons = [50, 25, 10]

dense1 = Dense(units=n_neurons[0], activation='relu')
dense2 = Dense(units=n_neurons[1], activation='relu')
dense3 = Dense(units=n_neurons[2], activation='softmax')

print("input shape", x.shape, '\n')
x = dense1(x)
W,B = dense1.get_weights()
print('W/B : {}/{}'.format(W.shape,B.shape))
print('after dense 1 : {}\n'.format(x.shape))

x = dense2(x)
W,B = dense2.get_weights()
print('W/B : {}/{}'.format(W.shape,B.shape))
print('after dense 2 : {}\n'.format(x.shape))

x = dense3(x)
W,B = dense3.get_weights()
print('W/B : {}/{}'.format(W.shape,B.shape))
print('after dense 3 : {}\n'.format(x.shape))

input shape (32, 10) 

W/B : (10, 50)/(50,)
after dense 1 : (32, 50)

W/B : (50, 25)/(25,)
after dense 2 : (32, 25)

W/B : (25, 10)/(10,)
after dense 3 : (32, 10)



## Shapes in the Loss Functions

In [25]:
from tensorflow.keras.losses import CategoricalCrossentropy

y = tf.random.uniform(minval=0, maxval=10, 
                      shape=(32,), 
                      dtype=tf.int32)
y = tf.one_hot(y, depth=10)

loss_object = CategoricalCrossentropy()
loss = loss_object(y,x)
print(loss.shape)
print(loss)

()
tf.Tensor(2.3012333, shape=(), dtype=float32)


# CNN Implementation

## implementation with Sequential Method

In [26]:
# Sequential model의 경우 모델 안에서 일어나는 일을 확인하기 어려움

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10,20,30]
n_dense_neurons = [50,30,10]
k_size, padding = 3, 'same'
activation='relu'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))
print(x.shape)
model = Sequential()
# Conv2D, MaxPooling2D
model.add(Conv2D(filters=n_conv_neurons[0],
                 kernel_size=k_size, 
                 padding=padding,
                 activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, 
                       strides=pool_strides))
model.add(Conv2D(filters=n_conv_neurons[1],
                 kernel_size=k_size, 
                 padding=padding,
                 activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, 
                       strides=pool_strides))
model.add(Conv2D(filters=n_conv_neurons[2],
                 kernel_size=k_size, 
                 padding=padding,
                 activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, 
                       strides=pool_strides))
model.add(Flatten())

# Dense
model.add(Dense(units=n_dense_neurons[0], 
                activation=activation))
model.add(Dense(units=n_dense_neurons[1], 
                activation=activation))
model.add(Dense(units=n_dense_neurons[2], 
                activation='softmax'))

predictions = model(x)
print(predictions.shape)

(4, 28, 28, 3)
(4, 10)


In [ ]:
# 반복문 활용
for n_conv_neuron in n_conv_neurons:
    model.add(Conv2D(filters=n_conv_neuron,
                 kernel_size=k_size, 
                 padding=padding,
                 activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))
model.add(Flatten())

for n_dense_neuron in n_dense_neurons:
    model.add(Dense(units=n_dense_neuron, activation=activation))
model.add(Dense(units=n_dense_neurons[-1], activation='softmax'))

## Implementation with Model Sub-classing

In [29]:
import tensorflow as tf

from tensorflow.keras.models import Model # 바뀐거

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10,20,30]
n_dense_neurons = [50,30,10]
k_size, padding = 3, 'same'
activation='relu'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()

        self.conv1 = Conv2D(filters=n_conv_neurons[0], kernel_size=k_size, 
                            padding=padding, activation=activation)
        self.conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
        self.conv2 = Conv2D(filters=n_conv_neurons[1], kernel_size=k_size, 
                            padding=padding, activation=activation)
        self.conv2_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
        self.conv3 = Conv2D(filters=n_conv_neurons[2], kernel_size=k_size, 
                            padding=padding, activation=activation)
        self.conv3_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
        self.flatten = Flatten()

        self.dense1 = Dense(units=n_dense_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_dense_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv1_pool(x)

        x = self.conv2(x)
        x = self.conv2_pool(x)

        x = self.conv3(x)
        x = self.conv3_pool(x)

        x = self.flatten(x)

        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)

        return x


N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10,20,30]
n_dense_neurons = [50,30,10]
k_size, padding = 3, 'same'
activation='relu'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))
model = TestCNN()
y = model(x)




(4, 28, 28, 3)
(4, 28, 28, 10)
(4, 14, 14, 10)
(4, 14, 14, 20)
(4, 7, 7, 20)
(4, 7, 7, 30)
(4, 3, 3, 30)
(4, 270)
(4, 50)
(4, 30)
(4, 10)


## Implementation with Sequential + Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model # for model subclassing
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, 
                            padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x

model = Sequential()
model.add(MyConv(n_conv_neurons[0]))
model.add(MyConv(n_conv_neurons[1]))
model.add(MyConv(n_conv_neurons[2]))
model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation=activation))
model.add(Dense(units=n_dense_neurons[1], activation=activation))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))


## Implementation with Model and Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model # for model subclassing
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, 
                            padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x



class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        self.conv1 = MyConv(n_conv_neurons[0])
        self.conv2 = MyConv(n_conv_neurons[1])
        self.conv3 = MyConv(n_conv_neurons[2])
        
        self.flatten = Flatten()

        self.dense1 = Dense(units=n_dense_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_dense_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = self.flatten(x)

        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)

        return x



In [30]:
import tensorflow as tf

from tensorflow.keras.models import Model # for model subclassing
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, 
                            padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x



class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        self.fe = Sequential()

        self.fe.add(MyConv(n_conv_neurons[0]))
        self.fe.add(MyConv(n_conv_neurons[1]))
        self.fe.add(MyConv(n_conv_neurons[2]))
        self.fe.add(Flatten())

        self.classifier = Sequential()
        self.classifier.add(Dense(units=n_dense_neurons[0], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[1], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[2], activation='softmax'))

    def call(self, x): # call 부분이 간소해짐. 
    
        x = self.fe(x)
        x = self.classfier(x)

        return x